# Build a Chatbot

Based on [**this LangChain tutorial**](https://python.langchain.com/v0.2/docs/tutorials/chatbot/)

# Overview

We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:
- [**Conversational RAG**](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/): Enable a chatbot experience over an external source of data.
- [**Agent**](https://python.langchain.com/v0.2/docs/tutorials/agents/): Build a chatbot that can take actions.

This tutorial will cover the basics which will be helpful for those two more advanced topics.

# Concepts

Here are a few of the high-level components we will be working with:
- [**Chat Models**](https://python.langchain.com/v0.2/docs/concepts/#chat-models). The chatbot interface is based around messages rather than raw text, and therefore is best suited to Chat Models rather than text LLMs.
- [**Prompt Templates**](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates), which simplify the process of assembling prompts that combine:
    - default messages,
    - user input,
    - chat history,
    - (optionally) additional retrieved content
- [**Chat History**](https://python.langchain.com/v0.2/docs/concepts/#chat-history), which allows a chatbot to "remember" past interactions and take them into account, when responding to follow-up questions
- Debugging and tracking your application using [**LangSmith**](https://python.langchain.com/v0.2/docs/concepts/#langsmith)

We'll cover how to fit the above components together to create a powerful conversational chatbot.

# Setup

In [1]:
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()

In [3]:
# To allow pretty printing
from rich import print as rprint

# Quickstart

In [4]:
from langchain_openai import ChatOpenAI

In [5]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
rprint(model)

ChatOpenAI(
    client=<openai.resources.chat.completions.Completions object at 0x76af8711d190>,
    async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x76af87551bd0>,
    openai_api_key=SecretStr('**********'),
    openai_proxy=''
)

Let's first use the model directly.

`ChatModel`s are instances of LangChain *Runnables*

In [11]:
model.__class__.__mro__

(langchain_openai.chat_models.base.ChatOpenAI,
 langchain_openai.chat_models.base.BaseChatOpenAI,
 langchain_core.language_models.chat_models.BaseChatModel,
 langchain_core.language_models.base.BaseLanguageModel,
 langchain_core.runnables.base.RunnableSerializable,
 langchain_core.load.serializable.Serializable,
 pydantic.v1.main.BaseModel,
 pydantic.v1.utils.Representation,
 langchain_core.runnables.base.Runnable,
 typing.Generic,
 abc.ABC,
 object)


This means **they expose a [standard interface](https://python.langchain.com/v0.1/docs/expression_language/interface/) for interacting with them**.

To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [12]:
from langchain_core.messages import HumanMessage

In [14]:
rprint(model.invoke([HumanMessage(content="Hi! I'm Bob.")]))

AIMessage(
    content='Hello Bob! How can I assist you today?',
    response_metadata={
        'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23},
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-fc74b463-c858-4170-98cf-b04d22be32fb-0',
    usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23}
)

> **API Reference: [HumanMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html)**

The model, on its own, **does not have any concept of state**.

For example, if you ask a followup question like this:

In [15]:
rprint(model.invoke([HumanMessage(content="What's my name?")]))

AIMessage(
    content="I'm sorry, but I do not have access to personal information such as your name. Is there anything else 
I can help you with?",
    response_metadata={
        'token_usage': {'completion_tokens': 28, 'prompt_tokens': 12, 'total_tokens': 40},
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-4ddf4dfa-9fae-4720-8638-142ad6894a20-0',
    usage_metadata={'input_tokens': 12, 'output_tokens': 28, 'total_tokens': 40}
)

Let's take a look at this example [**LangSmith trace**](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r)

We can see that **it doesn't take the previous conversation turn into context, and then cannot answer the last question**.

This makes for a terrible chatbot experience!

To get aroung this, we need to **pass the entire conversation history into the model**.

Let's see what happens in case we do that.

In [16]:
from langchain_core.messages import AIMessage

In [17]:
rprint(
    model.invoke([
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ])
)

AIMessage(
    content='Your name is Bob.',
    response_metadata={
        'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40},
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-aac8e5e9-d2ee-4482-9a3b-391145cc70cf-0',
    usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40}
)

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationnaly.

So, **how do we best implement this**?

# Message History

## Concepts

We can **use a Message History class to wrap our model and make it stateful**.

This will:
- **keep track of inputs and outputs of the model**,
- **store them in some datastore**.

Future interactions will then:
- **load those messages**,
- **pass them into the chain as part of the input**.

Let's see how to use this!

## Implementation

We can:
- import the relevant classes,
- set up our chain which wraps the model and adds in this message history.

**A key part here is the function we pass into as the `get_session_history`**.

This function **is expected to**:
- take in a `session_id`
- return a Message History object.

This `session_id`:
- is used to **distinguish between separate conversations**,
- should be **passed in as part of the config when calling the new chain**. (We'll show how to do that) 

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory  # Will be stored
from langchain_core.chat_history import BaseChatMessageHistory  # Only used for type hint
from langchain_core.runnables.history import RunnableWithMessageHistory  # Will be used to instanciate the wrapper

In [21]:
# Dict which will store the session_ids as keys
# and corresponding chat messages as values
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Chat history retriever, given a session_id"""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [22]:
type(with_message_history)

langchain_core.runnables.history.RunnableWithMessageHistory

In [24]:
with_message_history.__class__.__mro__

(langchain_core.runnables.history.RunnableWithMessageHistory,
 langchain_core.runnables.base.RunnableBindingBase,
 langchain_core.runnables.base.RunnableSerializable,
 langchain_core.load.serializable.Serializable,
 pydantic.v1.main.BaseModel,
 pydantic.v1.utils.Representation,
 langchain_core.runnables.base.Runnable,
 typing.Generic,
 abc.ABC,
 object)

> **API Reference**
- [**ChatMessageHistory**](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.ChatMessageHistory.html)
- [**BaseChatMessageHistory**](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html)
- [**RunnableWithMessageHistory**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

> **IMPORTANT**
> 
> **We now need to create a `config` that we pass into the runnable every time**.

This config contains information that is not part of the input directly, but is still useful.

In this case, we want to include a `session_id`. This should look like...

In [25]:
config = {"configurable": {"session_id": "ObiwanKenobi"}}

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob.")],
    config=config,
)
response.content

'Hello Bob, nice to meet you! How are you doing today?'

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Bob.'

Great! Our chatbot is now able to "remember" things about us.

If we change the config to reference a different `session_id`, we can start a conversation from fresh.

In [29]:
config = {"configurable": {"session_id": "Luke"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

"I'm sorry, I do not have the ability to know your name."

However, we can still go back to the original conversation, as we're persisting it in `store`.

In [30]:
config = {"configurable": {"session_id": "ObiwanKenobi"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Bob.'

This is how we can support a chatbot having conversations with many users!

Right now, all we-ve done is add a simple persistence layer around the model.

We can start to make the more complicated and personalized by adding in a **prompt template**.